In [1]:
!git clone https://github.com/logIoT/2024-distillog.git

Cloning into '2024-distillog'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 169 (delta 64), reused 151 (delta 46), pack-reused 0
Receiving objects: 100% (169/169), 38.98 MiB | 21.50 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [4]:
%cd 2024-distillog/Spirit5M-kd-chronological

/kaggle/working/2024-distillog/Spirit5M-kd-chronological


In [5]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.nn.modules.module import Module
from tqdm import tqdm
import math
import csv
from time import time
from torchinfo import summary
from utils import save_model
from utils import DistilLog
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

fi = pd.read_csv('../datasets/Spirit/vector.csv', header = None)
vec = []
vec = fi
vec = np.array(vec)

logs_series = pd.read_csv('../datasets/Spirit/chronological_train.csv', header = None)
logs_series = logs_series.values

train_total = len(logs_series)
split = 2000
sub = int(train_total/split)


def model_train(model, learning_rate, num_epochs):
    criterion = nn.CrossEntropyLoss()
    #summary(model, input_size=(50, 50, 30))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for i in trange(0, split):
            train_x, train_y = read_data(input_size, sequence_length, i)
            train_loader = load_data(train_x, train_y, batch_size)
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output, _ = model(data)
                loss = criterion(output, target)
                total_loss += loss.item()
                loss.backward()
                optimizer.step()


        print(f'Epoch {epoch}/{num_epochs}, total loss = {total_loss}')

    return model

In [6]:
print(sub)

40


In [7]:
def mod(l, n):
    """ Truncate or pad a list """
    r = l[-1 * n:]
    if len(r) < n:
        r.extend(list([0]) * (n - len(r)))
    return r

def read_data(input_size, sequence_length, i):
    if i!=split-1:
        label = logs_series[i*sub:(i+1)*sub,1]
        logs_data = logs_series[i*sub:(i+1)*sub,0]
    else:
        label = logs_series[i*sub:,1]
        logs_data = logs_series[i*sub:,0]
    logs = []
    for i in range(0, len(logs_data)):
        ori_seq = [
            int(eventid) for eventid in logs_data[i].split()]
        seq_pattern = mod(ori_seq, sequence_length)
        vec_pattern = []

        for event in seq_pattern:
            if event == 0:
                vec_pattern.append([-1] * input_size)
            else:
                vec_pattern.append(vec[event - 1])
        logs.append(vec_pattern)
    logs = np.array(logs)
    train_x = logs
    train_y = np.array(label)
    train_x = np.reshape(train_x, (train_x.shape[0], -1, input_size))
    train_y = train_y.astype(int)
    return train_x, train_y


def load_data(train_x, train_y, batch_size):
    tensor_x = torch.Tensor(train_x)
    tensor_y = torch.from_numpy(train_y)
    train_dataset = TensorDataset(tensor_x, tensor_y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    return train_loader

In [8]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.nn.modules.module import Module
from tqdm import tqdm, trange
import math
import csv
from time import time
from torchinfo import summary
from utils import save_model, train
from utils import DistilLog
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2
batch_size = 50
learning_rate = 0.0005
hidden_size = 128
input_size = 300
sequence_length = 50
num_layers = 2

train_path = '../datasets/Spirit/chronological_train.csv'
save_teacher_path = '../datasets/Spirit/model/chronological_teacher(300).pth'
save_noKD_path = '../datasets/Spirit/model/chronological_noKD(300).pth'

Teacher = DistilLog(input_size, hidden_size, num_layers , num_classes, is_bidirectional=False).to(device)
noKD = DistilLog(input_size = input_size, hidden_size = 4, num_layers = 1, num_classes = num_classes, is_bidirectional=False).to(device)
#summary(Teacher, input_size=(50, 50, 30))

#train_x, train_y = read_data(train_path, input_size, sequence_length)
#train_loader = load_data(train_x, train_y, batch_size)

#Teacher = train(Teacher, train_loader, learning_rate = 0.0003, num_epochs = 400)
#save_model(Teacher, save_teacher_path)



/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
Teacher = model_train(Teacher, learning_rate = 0.0003, num_epochs = 100)
save_model(Teacher, save_teacher_path)
#noKD = model_train(noKD, learning_rate = 0.0003, num_epochs = 100)

100%|██████████| 2000/2000 [00:18<00:00, 105.77it/s]


Epoch 0/100, total loss = 319.39097813849367


100%|██████████| 2000/2000 [00:17<00:00, 114.73it/s]


Epoch 1/100, total loss = 129.28962522517259


100%|██████████| 2000/2000 [00:18<00:00, 109.62it/s]


...

Epoch 97/100, total loss = 28.52617902707314


100%|██████████| 2000/2000 [00:17<00:00, 113.76it/s]


Epoch 98/100, total loss = 28.671905936694976


100%|██████████| 2000/2000 [00:17<00:00, 113.31it/s]


Epoch 99/100, total loss = 32.10351134201859


In [10]:
Student = DistilLog(input_size = input_size, hidden_size = 4, num_layers = 1, num_classes = num_classes, is_bidirectional=False).to(device)

In [11]:
import torch.nn.functional as F

train_total = len(logs_series)
split = 2000
sub = int(train_total/split)

def train_step(
    Teacher,
    Student,
    optimizer,
    student_loss_fn,
    divergence_loss_fn,
    temp,
    alpha,
    epoch,
    device
):
    losses = []
    for i in trange(split):
      train_x, train_y = read_data(input_size, sequence_length, i)
      train_loader = load_data(train_x, train_y, batch_size)
      for batch_idx, (data, targets) in enumerate(train_loader):
          data, targets = data.to(device), targets.to(device)
          # forward
          with torch.no_grad():
              teacher_preds, _ = Teacher(data)

          student_preds, __ = Student(data)
          student_loss = student_loss_fn(student_preds, targets)

          ditillation_loss = divergence_loss_fn(
              F.softmax(student_preds / temp, dim=1),
              F.softmax(teacher_preds / temp, dim=1)
          )
          loss = alpha * student_loss + (1 - alpha) * ditillation_loss
          losses.append(loss.item())

          # backward
          optimizer.zero_grad()
          loss.backward()

          optimizer.step()

    avg_loss = sum(losses) / len(losses)
    return avg_loss

def teach(epochs, Teacher, Student, temp=7, alpha=0.3):
  Teacher = Teacher.to(device)
  Student = Student.to(device)
  student_loss_fn = nn.CrossEntropyLoss()
  divergence_loss_fn = nn.KLDivLoss(reduction="batchmean")
  optimizer = torch.optim.Adam(Student.parameters(), lr=0.01)

  Teacher.eval()
  Student.train()
  for epoch in range(epochs):
      loss = train_step(
          Teacher,
          Student,
          optimizer,
          student_loss_fn,
          divergence_loss_fn,
          temp,
          alpha,
          epoch,
          device
      )

      print(f"Loss:{loss:.2f}")


In [12]:

teach(epochs=100, Teacher=Teacher, Student=Student, temp=7, alpha=0.3)

100%|██████████| 2000/2000 [00:19<00:00, 102.38it/s]


Loss:-0.71


100%|██████████| 2000/2000 [00:19<00:00, 102.28it/s]


Loss:-0.77


100%|██████████| 2000/2000 [00:19<00:00, 102.59it/s]


...

Loss:-0.81


100%|██████████| 2000/2000 [00:18<00:00, 107.12it/s]


Loss:-0.81


100%|██████████| 2000/2000 [00:18<00:00, 106.23it/s]

Loss:-0.81


In [13]:
from utils import load_data, load_model
test_path = '../datasets/Spirit/chronological_test.csv'
test_split = 50

fi = pd.read_csv('../datasets/Spirit/vector.csv', header = None)
vec = []
vec = fi
vec = np.array(vec)

test_logs_series = pd.read_csv(test_path)
test_logs_series = test_logs_series.values
test_total = len(test_logs_series)
sub = int(test_total/test_split)



def mod(l, n):
    """ Truncate or pad a list """
    r = l[-1*n:]
    if len(r) < n:
        r.extend(list([0]) * (n - len(r)))
    return r

def load_test(i):problems
    if i!=test_split-1:
        label = test_logs_series[i*sub:(i+1)*sub,1]
        logs_data = test_logs_series[i*sub:(i+1)*sub,0]
    else:
        label = test_logs_series[i*sub:,1]
        logs_data = test_logs_series[i*sub:,0]
    logs = []

    for logid in range(0,len(logs_data)):
        ori_seq = [
            int(eventid) for eventid in logs_data[logid].split()]
        seq_pattern = mod(ori_seq, sequence_length)
        vec_pattern = []

        for event in seq_pattern:
            if event == 0:
                vec_pattern.append([-1]*input_size)
            else:
                vec_pattern.append(vec[event-1])
        logs.append(vec_pattern)
    logs = np.array(logs)
    train_x = logs
    train_y = label
    train_x = np.reshape(train_x, (train_x.shape[0], -1, input_size))
    train_y = train_y.astype(int)
    return train_x, train_y



def test(model, criterion = nn.CrossEntropyLoss()):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        TP = 0
        FP = 0
        FN = 0
        TN = 0
        for i in trange(test_split):
        #################################################
            test_x, test_y = load_test(i)
            test_loader = load_data(test_x, test_y, batch_size)
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output, _ = model(data)
                test_loss += criterion(output, target) # sum up batch loss

                output = torch.sigmoid(output)[:, 0].cpu().detach().numpy()
                predicted = (output < 0.2).astype(int)
                target = np.array([y.cpu() for y in target])

                TP += ((predicted == 1) * (target == 1)).sum()
                FP += ((predicted == 1) * (target == 0)).sum()
                FN += ((predicted == 0) * (target == 1)).sum()
                TN += ((predicted == 0) * (target == 0)).sum()
        P = 100 * TP / (TP + FP)
        R = 100 * TP / (TP + FN)
        F1 = 2 * P * R / (P + R)
        accuracy = 100 * (TP + TN)/(TP + TN + FP + FN)
        #MCC = 100*(TP*TN + FP*FN)/math.sqrt((TP+FP)*(TN+FN)*(TN+FP)*(TP+FN))
    return accuracy, test_loss, P, R, F1, TP, FP, TN, FN

#Teacher = load_model(Teacher, save_teacher_path)


start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(Teacher, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('Result of testing teacher model')
print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%' .format(P, R, F1))


100%|██████████| 50/50 [00:04<00:00, 11.07it/s]

Result of testing teacher model
false positive (FP): 32, false negative (FN): 346, true positive (TP): 160, true negative (TN): 19461
Test set: Average loss: 0.0000, Accuracy: 98.11%). Total time = 4.524524688720703
Precision: 83.333%, Recall: 31.621%, F1-measure: 45.845%


In [14]:
start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(Student, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('Result of testing student model')
print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%' .format(P, R, F1))


100%|██████████| 50/50 [00:03<00:00, 13.13it/s]

Result of testing student model
false positive (FP): 3, false negative (FN): 364, true positive (TP): 142, true negative (TN): 19490
Test set: Average loss: 0.0001, Accuracy: 98.16%). Total time = 3.8176121711730957
Precision: 97.931%, Recall: 28.063%, F1-measure: 43.625%
